In [33]:
import pandas as pd
import pyprind
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.io as pio
import nltk
nltk.download('wordnet')
pio.renderers.default = 'vscode'

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ryan_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
file_path = "C:/Users/ryan_/OneDrive/Desktop/Suicide Watch Detection/src/Data/cleaned_data.csv"

# Define column names based on the first line of the file
data = pd.read_csv(file_path)



data.rename(columns={'class': 'label'}, inplace=True)


df = pd.DataFrame(data)


df.head()

,text,label
0,"['wife', 'threaten', 'suicide', 'recently', 'l...",1
1,"['weird', 'get', 'affect', 'come', 'someone', ...",0
2,"['finally', 'almost', 'never', 'hear', 'bad', ...",0
3,"['need', 'help', 'just', 'help', 'cry', 'hard']",1
4,"['lose', 'hello', 'name', 'struggle', 'year', ...",1


## RNN


#### Creating datasets

In [89]:

# Step 3: Define Custom Dataset Class
class CustomDataset():
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx, 0]  # Assuming text is in the first column
        label = self.dataframe.iloc[idx, 1]  # Assuming label is in the second column
        return text, label


In [35]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

# Load the CSV file into a DataFrame, specifying the delimiter
df = pd.read_csv(r'C:\Users\ryan_\OneDrive\Desktop\Suicide Watch Detection\src\Data\cleaned_data.csv', delimiter=',', header=None)  # Assuming no header

df = df.reset_index(drop=True)
# Rename the class column to "label"
df = df.rename(columns={0: 'text', 1: 'label'})

# Split the data into train, validation, and test sets
train_valid_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, valid_df = train_test_split(train_valid_df, test_size=0.25, random_state=42)  # Splitting 80% for train and 20% for validation

# Process tokens to remove extra characters (such as commas, single quotes, and square brackets)
def preprocess_tokens(sentence):
    return [word.strip(",'[]") for word in sentence.split()]

# Create a counter to count word frequencies
word_counter = Counter()

# Count word frequencies from the training data
for sentence in train_df['text']:
    word_counter.update(preprocess_tokens(sentence))

# Create a vocabulary ordered by word frequency
vocab_ordered = {'<pad>': 0, '<unk>': 1}
word_index = len(vocab_ordered)
for word, count in word_counter.most_common():
    vocab_ordered[word] = word_index
    word_index += 1

# Optionally, you can save the vocab to a file for later use
with open('vocab_ordered.txt', 'w') as f:
    for word, index in vocab_ordered.items():
        f.write(f"{word}: {index}\n")

# Now you have train_df, valid_df, test_df DataFrames with the label column renamed
# You can proceed with tokenization, building vocabulary, and encoding labels as needed


In [36]:
# View a sample of your training data
print("Sample of training data:")
print(train_df.head())

# View a sample of your validation data
print("\nSample of validation data:")
print(valid_df.head())

# View a sample of your test data
print("\nSample of test data:")
print(test_df.head())

# View your vocabulary
print("\nVocabulary:")
print(vocab_ordered)

Sample of training data:
                                                     text label
77983   ['one', 'masturbate', 'class', 'dare', 'mastur...     0
116775  ['fake', 'smile', 'fake', 'enjoy', 'work', 'fa...     1
18248   ['want', 'kill', 'could', 'die', 'know', 'woul...     1
194756  ['yes', 'finally', 'basically', 'girl', 'gon',...     0
98811   ['chop', 'screw', 'slow', 'slow', 'reverb', 's...     0

Sample of validation data:
                                                     text label
118219  ['defend', 'try', 'convince', 'people', 'endur...     1
195809  ['anyone', 'want', 'combat', 'zombie', 'outbre...     0
118537  ['want', 'live', 'want', 'escape', 'nowhere', ...     1
224330  ['auto', 'remove', 'post', 'post', 'make', 'pa...     0
152758  ['get', 'well', 'life', 'downward', 'spiral', ...     1

Sample of test data:
                                                     text label
74414   ['depressed', 'year', 'get', 'help', 'last', '...     1
149516  ['feel', 'sad', 'year

# THIS IS THE PROBLEM!!!

In [37]:
text_pipeline = lambda x: [vocab_ordered[word] if word in vocab_ordered else vocab_ordered['<unk>'] for word in preprocess_tokens(x)]
label_pipeline = lambda x: int(x)

    


# TASKS FOR Today:
### Build 3 RNN Models using these three types of recurrent layers
#### RNN
#### LSTM
#### GRU


In [63]:
import torch
from torch.utils.data import DataLoader
def collate_batch(batch):
    text_list, label_list, length = [], [], []
    for text, label in batch:
        label_list.append(label_pipeline(label))
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
        length.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.float32)
    length = torch.tensor(length)
    padded_text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True)
    
    # Create a dictionary containing the processed text, labels, and lengths
    batch_dict = {
        'text': padded_text_list,
        'label': label_list,
        'length': length
    }
    return batch_dict

# Assuming train_data is properly defined
train_dataloader = DataLoader(custom_dataset, batch_size=4, shuffle=True, collate_fn=collate_batch)

In [90]:
train_df = CustomDataset(train_df)
valid_df = CustomDataset(valid_df)
test_df = CustomDataset(test_df)

In [91]:
batch_size = 32
train_dl = DataLoader(train_df, batch_size=batch_size,shuffle=True,collate_fn=collate_batch)
valid_dl = DataLoader(valid_df, batch_size=batch_size,shuffle=False,collate_fn=collate_batch)
test_dl = DataLoader(test_df, batch_size=batch_size,shuffle=False,collate_fn=collate_batch)



In [66]:
import torch
import torch.nn as nn
embedding = nn.Embedding(num_embeddings=word_index, embedding_dim=100, padding_idx=0)

In [67]:

import numpy as np
import torch

class SGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        
    def step(self, rnn_model, gradients):
        for param_name, gradient in gradients.items():
            setattr(rnn_model, param_name, getattr(rnn_model, param_name) - self.learning_rate * gradient)


class RNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Random Initial Weights
        self.Wx = np.random.randn(hidden_size, input_size) * 0.01
        self.Wxy = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Wyz = np.random.randn(output_size, hidden_size) * 0.01
        self.Bh = np.zeros((hidden_size, 1))
        self.Bo = np.zeros((output_size, 1))
        
        self.learning_rate = learning_rate
        self.optimizer = None

    def forward(self, inputs, h_prev):
        self.a = np.dot(self.Wx, inputs) + np.dot(self.Wxy, h_prev) + self.Bh
        self.h = np.tanh(self.a)
        self.y = np.dot(self.Wyz, self.h) + self.Bo
        return self.y, self.h

    def backward(self, d_y, h, h_prev, inputs):
        d_Wyz = np.dot(d_y, self.h.T)
        d_Bo = d_y
        d_h = np.dot(self.Wyz.T, d_y)
        d_a = np.dot(1 - self.h ** 2, d_h)
        d_Bh = d_a
        d_Wxy = np.dot(d_a, h_prev.T)
        d_Wx = np.dot(d_a, inputs.T)
        d_h_prev = np.dot(self.Wxy.T, d_a)
        
        gradients = {
            'd_Wx': d_Wx,
            'd_Wxy': d_Wxy,
            'd_Wyz': d_Wyz,
            'd_Bh': d_Bh,
            'd_Bo': d_Bo,
            'd_h_prev': d_h_prev
        }
        return gradients
    
    def train_step(self, inputs, targets, h_prev):
        # Forward pass
        y_pred, hidden_state = self.forward(inputs, h_prev)

        # Calculate loss
        loss = np.mean((y_pred - targets)**2)

        # Backward pass
        gradients = self.backward(2 * (y_pred - targets), hidden_state, h_prev, inputs)

        # Update weights
        self.optimizer.step(self, gradients)

        return loss

# Example usage:
input_size = 10
hidden_size = 64
output_size = 2
learning_rate = 0.01

rnn_model = RNN(input_size, hidden_size, output_size, learning_rate)




In [95]:
input_size = 10
hidden_size = 64
output_size = 1
learning_rate = 0.01

rnn_model = RNN(input_size, hidden_size, output_size, learning_rate)
rnn_model.optimizer = SGD(learning_rate)
def train_model(dataloader):  
    total_acc, total_loss = 0.0, 0.0
    for i, batch in enumerate(dataloader):
        print(f"Batch Index: {i}")
        text_batch = batch['text']
        label_batch = batch['label']
        lengths = batch['length']
        
        h_prev = np.zeros((hidden_size, text_batch.shape[0]))  # Initialize hidden state
        
        loss = rnn_model.train_step(text_batch, label_batch, h_prev)
        
        total_loss += loss * text_batch.shape[0]
        
        pred = rnn_model.forward(text_batch, h_prev)[0]
        total_acc += ((pred >= 0.5).astype(float) == label_batch).astype(float).sum().item()
        
    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)


In [97]:
rnn_model = RNN(input_size = 32, hidden_size = 186, output_size = 2)
learning_rate = 0.01
num_epochs = 10




In [78]:
def evaluate(dataloader):  
    total_acc, total_loss = 0.0, 0.0
    for i, batch in enumerate(dataloader):
        print(f"Batch Index: {i}")
        text_batch = batch['text']
        label_batch = batch['label']
        lengths = batch['length']
        
        h_prev = np.zeros((hidden_size, text_batch.shape[0]))  # Initialize hidden state
        
        loss = rnn_model.train_step(text_batch, label_batch, h_prev)
        
        total_loss += loss * text_batch.shape[0]
        
        pred = rnn_model.forward(text_batch, h_prev)[0]
        total_acc += ((pred >= 0.5).astype(float) == label_batch).astype(float).sum().item()
        
    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)


In [96]:

torch.manual_seed(1)
for epoch in range(num_epochs):
    acc_train, loss_train = train_model(train_dl)
    

Batch Index: 0


ValueError: shapes (64,10) and (32,186) not aligned: 10 (dim 1) != 32 (dim 0)